# Term project - MNIST visualization<br>
Deep neural network 구현 및 Visualization과 이를 통한 Neural network에 대한 이해 향상


## MNIST dataset 불러 오기 및 시각화

1. data에 대한 pca(2d라? 겹치는 경유 확인가능) + tsne결과 확인하기
    둘의 차이 확인하기
2. layer들의 dimension 확인

• Input image에 대한 PCA, t-SNE 결과
• hidden vector (a[0], z[1], a[1], z[2], a[2])에 대한 PCA, ,t-SNE결과
    ->  총 10장
• 결과 분석 (자유롭게)


In [90]:
from __future__ import print_function

import torch
from torch import nn
from torch import optim
import numpy as np

import time

# dataset
import torchvision 
from torchvision import datasets
from torchvision import transforms

import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# sklearn
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns


%matplotlib inline

## Hyperparameter
modeling 과정에서 직접 설정하는 값입니다.<br>
- batch_size
- learning rate : 학습률(gradient를 통해 weight를 이동, 변화시키는 정도)
- epoch_nums : 학습을 반복하는 횟수
- batch size : 한번의 batch마다 주어지는 data sample의 size (batch: 나누어진 dataset)
- momentum : weight의 update과정에서 관성, 가속도를 주어 감소, 증가하는 방향으로 더 많은 변화를 준다.

In [91]:
batch_size = 12
learning_rate = 0.01
epoch_num = 10

## MNIST dataset
### torchvision.datasets.MNIST = (root="", train="", dawnload="",trasform ="")
root = "data의 경로" <br>
train = True(학습용 데이터) or False(test용 데이터) <br>
tramsform - 데이터 형태 <br>
download = True(MNIST dataset이 없으면 다운로드한다.) <br>

In [92]:
train_data = datasets.MNIST(root= '..\datasets', train= True, download= True, transform= transforms.ToTensor())
test_data = datasets.MNIST(root= '..\datasets', train= False, download= True, transform= transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

# Deep Neural Network
input layer와 ouput layer사이에 hidden layer를 2개이상 지닌 neural network을 의미합니다.
- 분류와 수치예측을 주목적으로 하여 이미지 학습과 문자인식에 유용하게 사용되고 있는 신경망입니다.

## Using squential 
squential하여 내부  module에 접급하지 않는다.

## Using ModuleList
Module을 리스트 형태로 담을 때 사용합니다. ModuleList에서는 module하나하나에 접근이 가능하다.

In [93]:
# Using ModuleList
class DeepNN_ML(nn.Module):
    def __init__(self):
        super(DeepNN_ML, self).__init__()

        self.in_dim = 28 * 28 # MNIST
        self.out_dim = 10 # 0 ~ 9
        self.az = [ [] for i in range(10)]
        
        self.list = nn.ModuleList([
            nn.Linear(in_features = self.in_dim, out_features = 512),
            nn.ReLU(),
            nn.Linear(in_features = 512, out_features = 256),
            nn.ReLU(),
            nn.Linear(in_features = 256, out_features = 128),
            nn.ReLU(),
            nn.Linear(in_features = 128, out_features = 64),
            nn.ReLU(),
            nn.Linear(in_features = 64, out_features = self.out_dim)
        ])


    def forward(self, x, V):
        fw = x.view(-1, self.in_dim)
        if V > 5000: 
            for i, module in enumerate(self.list):
                fw = module(fw)
        else:
            self.az[0].append(fw.tolist())
            for i, module in enumerate(self.list):
                fw = module(fw)
                self.az[i+1].append(fw.tolist())
        return fw



In [94]:
model_ML = DeepNN_ML()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ML.parameters(), lr = learning_rate)
label_list = []
V = (60000/batch_size) * epoch_num
for epoch in range(epoch_num):
    running_loss = 0.0
    for i, data in enumerate(train_loader,0):
        # get the inputs: data is a list of [input, label]
        inputs, labels = data
        if epoch == epoch_num-1:
            label_list.append(labels.tolist())
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward, backward, optimizer
        outputs = model_ML(inputs, V)
        loss = criterion(outputs, labels) # loss
        loss.backward() # backward
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if (i+1) % 2000 == 0 : # print every 2000 mini-batch
            print("[%d, %5d] loss: %.3f" % (epoch+1, i+1, running_loss/2000) )
            running_loss = 0.0
        V -= 1
print("Finished training")


[1,  2000] loss: 2.209
[1,  4000] loss: 0.665
[2,  2000] loss: 0.305
[2,  4000] loss: 0.224
[3,  2000] loss: 0.157
[3,  4000] loss: 0.140
[4,  2000] loss: 0.109
[4,  4000] loss: 0.099
[5,  2000] loss: 0.080
[5,  4000] loss: 0.077
[6,  2000] loss: 0.064
[6,  4000] loss: 0.061
[7,  2000] loss: 0.045
[7,  4000] loss: 0.050
[8,  2000] loss: 0.039
[8,  4000] loss: 0.041
[9,  2000] loss: 0.031
[9,  4000] loss: 0.030
[10,  2000] loss: 0.023
[10,  4000] loss: 0.026
Finished training
